In [3]:
import pandas as pd
import numpy as np
import os
from IPython.display import display
from tqdm import tqdm

from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import callbacks

from sklearn.model_selection import GroupKFold

In [4]:
class CFG:
   debug=True
   dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
   models_dir="./models/" if debug else ""
   target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
   'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']
   n_folds=5

In [5]:
train=pd.read_csv(f"{CFG.dataset_dir}train.csv")
y=train[CFG.target_cols]

In [6]:
npz=np.load("../input/effnet_best_output.npz")
features_list=[npz[uid] for uid in tqdm(train["StudyInstanceUID"])]
features=np.array(features_list)

100%|██████████| 30083/30083 [00:55<00:00, 546.54it/s]


In [7]:
def get_fold(train):
    fold=train.copy()
    splitter=GroupKFold(n_splits=CFG.n_folds)
    for n,(train_idx,val_idx) in enumerate(splitter.split(train,groups=train["PatientID"])):
        fold.loc[val_idx,"folds"]=n
    fold["folds"]=fold["folds"].astype(int)
    return fold

fold=get_fold(train)

In [8]:
def create_model():
    dense_model=keras.Sequential([
        keras.layers.Dense(len(CFG.target_cols),input_shape=(2560,),activation="sigmoid")
    ])
    adam=keras.optimizers.Adam(learning_rate=1e-4)
    dense_model.compile(optimizer=adam,loss="binary_crossentropy",metrics=[keras.metrics.AUC(multi_label=True,name="auc")])
    return dense_model

In [9]:
early_stopping=callbacks.EarlyStopping(monitor="val_auc",patience=50,mode="max")
lr_reducer=callbacks.ReduceLROnPlateau(monitor="val_auc",patience=7,min_lr=1e-6,mode="max",factor=0.5,verbose=1)

In [10]:
save_dir=f"{CFG.models_dir}effnet_best_dense/"
os.makedirs(save_dir,exist_ok=True)

for n in range(CFG.n_folds):
    train_idx=(fold["folds"]!=n)
    val_idx=(fold["folds"]==n)
    X_train=features[train_idx]
    X_val=features[val_idx]
    y_train=y[train_idx]
    y_val=y[val_idx]

    dense_model=create_model()
    checkpoint=callbacks.ModelCheckpoint(f"{save_dir}ckpt_{n}",monitor="val_auc",mode="max",
        save_weights_only=True,save_best_only=True)

    dense_model.fit(X_train,y_train,epochs=100,shuffle=True,validation_data=(X_val,y_val),callbacks=[early_stopping,lr_reducer,checkpoint])
    dense_model.load_weights(f"{save_dir}ckpt_{n}")
    score=dense_model.evaluate(X_val,y_val,verbose=0)
    print(f"\nbest score: {score}")

ss: 0.1008 - auc: 0.9717 - val_loss: 0.0974 - val_auc: 0.9678
Epoch 21/100
753/753 [==============================] - 2s 3ms/step - loss: 0.1007 - auc: 0.9717 - val_loss: 0.0973 - val_auc: 0.9678
Epoch 22/100
753/753 [==============================] - 2s 3ms/step - loss: 0.1007 - auc: 0.9716 - val_loss: 0.0974 - val_auc: 0.9677
Epoch 23/100
753/753 [==============================] - 2s 3ms/step - loss: 0.1007 - auc: 0.9718 - val_loss: 0.0974 - val_auc: 0.9678
Epoch 24/100
753/753 [==============================] - 2s 3ms/step - loss: 0.1007 - auc: 0.9716 - val_loss: 0.0973 - val_auc: 0.9679
Epoch 25/100
753/753 [==============================] - 2s 3ms/step - loss: 0.1007 - auc: 0.9716 - val_loss: 0.0973 - val_auc: 0.9678
Epoch 26/100
753/753 [==============================] - 2s 3ms/step - loss: 0.1007 - auc: 0.9717 - val_loss: 0.0974 - val_auc: 0.9678
Epoch 27/100
738/753 [============================>.] - ETA: 0s - loss: 0.1006 - auc: 0.9718
Epoch 00027: ReduceLROnPlateau reducing l